In [20]:
from pytube import Stream,Playlist
from pytube import YouTube
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import requests
from tqdm import tqdm
import os

In [21]:
def get_channel_playlists(channel):
    driver = webdriver.Chrome()
    driver.get(f'https://www.youtube.com/@{channel}/playlists')
    soup = BeautifulSoup(driver.page_source,'html.parser')
    driver.close()
    return {channel:[{'id':p.get('href').split('list=')[1], 'name':p.getText(strip=True)} for p in soup.select("#video-title")]} # extract playlist_ids

def get_playlist_videos(playlist_id):
    return Playlist(f'https://www.youtube.com/playlist?list={playlist_id}').video_urls

def get_playlists_videos(playlist_ids):
    return {playlist_id:get_playlist_videos(playlist_id) for playlist_id in playlist_ids}

def save_video(url):
    def progress_callback(stream: Stream, data_chunk: bytes, bytes_remaining: int) -> None:
        #pbar.update(len(data_chunk))
        pass
    yt = YouTube(url, on_progress_callback=progress_callback)
    video = yt.streams.get_highest_resolution() #480 if not then 720 then 360,144
    if not os.path.exists(f'./dl_files/{video.default_filename}'):
        print(f'file name: {video.default_filename}\nsize: {video.filesize_mb}\nquality:{video.resolution}')
        #pbar = tqdm(total=video.filesize, unit="bytes")
        try:
            video.download('./dl_files')
            print(f'downloaded {video.default_filename}')
        except Exception as e:
            if os.path.exists(f'./dl_files/{video.default_filename}'):
                os.remove(f'./dl_files/{video.default_filename}')
            raise e
        #pbar.close()
    else:
        print(f"video exists : {video.default_filename}")

def save_videos(videos_url:list):
    with ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(save_video, videos_url)

In [ ]:
playlists = get_channel_playlists('vtoo_learn')
playlists_ids = [playlist['id'] for playlist in playlists['vtoo_learn']]
playlists_videos = get_playlists_videos(playlists_ids)

playlists_df = pd.DataFrame(playlists['vtoo_learn'])
rows = []
for key, values in playlists_videos.items():
    for value in values:
        rows.append({'id': key, 'value': value})
playlists_videos_df = pd.DataFrame(rows)
playlists_videos_df = playlists_videos_df.merge(playlists_df, on='id').rename({'value':'video_url'}, axis=1).reset_index(drop=True)

In [23]:
playlists

{'vtoo_learn': [{'id': 'PLlaijeQ_sCnJf0Zg7iQGPNiZIuqbT9i9a&pp=iAQB',
   'name': 'پروژه'},
  {'id': 'PLlaijeQ_sCnIdja5twIr7fqSD7sHcC2t7&pp=iAQB',
   'name': 'مدارهای الکتریکی'},
  {'id': 'PLlaijeQ_sCnIfeNTPSsRncgMTATLhHN5g&pp=iAQB',
   'name': 'آموزش مولتی سیم'},
  {'id': 'PLlaijeQ_sCnI1opXUssq3nGhYnvFVLlvp&pp=iAQB', 'name': 'آموزش یوتیوب'},
  {'id': 'PLlaijeQ_sCnLkp0VCM61fqvzkxgtpvUIh&pp=iAQB',
   'name': 'دانشکده باهنر شیراز'},
  {'id': 'PLlaijeQ_sCnLzDp70OzdqV06vn6r-x7qs&pp=iAQB',
   'name': 'آموزش نرم افزار موبایل'},
  {'id': 'PLlaijeQ_sCnL8pwhyna4dCUmJx5oKL9Sg&pp=iAQB',
   'name': 'اینترنت ماهواره ای'},
  {'id': 'PLlaijeQ_sCnJTB3TTxX47-7ZVa7JINRoC&pp=iAQB', 'name': 'پادکست'},
  {'id': 'PLlaijeQ_sCnKIeDo3gCTQjMaHQsEUcSef&pp=iAQB',
   'name': 'دیتاشیت خوانی Datasheet'},
  {'id': 'PLlaijeQ_sCnKRwxW1tg9SqIlffj3MiRXy&pp=iAQB',
   'name': 'آموزش الکترونیک'},
  {'id': 'PLlaijeQ_sCnK_P2RbVXbbCESAmadk4RMT&pp=iAQB', 'name': 'متفرقه'},
  {'id': 'PLlaijeQ_sCnIXWcqDm_9t1z_JIj1ZsXKj&pp=iAQB', 'n

In [26]:
videos_to_download = playlists_videos_df[playlists_videos_df['name'].isin(['آموزش سخت افزار موبایل'])]['video_url'].values
videos_to_download

array(['https://www.youtube.com/watch?v=w2beXvm0uhI',
       'https://www.youtube.com/watch?v=H_gAkKb30fU',
       'https://www.youtube.com/watch?v=fZtbn6oohvc',
       'https://www.youtube.com/watch?v=TUr0lN6iLsA',
       'https://www.youtube.com/watch?v=WsgA8wE8YLA',
       'https://www.youtube.com/watch?v=3CSI7V5LHdI',
       'https://www.youtube.com/watch?v=dtZ8VyzlFcQ',
       'https://www.youtube.com/watch?v=DyXs3WybZgQ',
       'https://www.youtube.com/watch?v=pVXYDAiAbgI',
       'https://www.youtube.com/watch?v=v9Wl4zxBdxU',
       'https://www.youtube.com/watch?v=qgF23ErBXsE',
       'https://www.youtube.com/watch?v=ZWCDCFCv7A4',
       'https://www.youtube.com/watch?v=16Xx7lQSzmk',
       'https://www.youtube.com/watch?v=khXKSiFZZUY',
       'https://www.youtube.com/watch?v=S7YuWTezWYM',
       'https://www.youtube.com/watch?v=HqIMkj0snCY',
       'https://www.youtube.com/watch?v=7zG1h8606ek',
       'https://www.youtube.com/watch?v=dgD9uzHMU5c',
       'https://www.youtube.

In [27]:
save_videos(videos_to_download)

file name: آموزش تعمیرات موبایل جلسه ی 1.mp4
size: 114.246
quality:720p
file name: آموزش تعمیرات موبایل جلسه ی 3.mp4
size: 169.352
quality:720p
file name: آموزش تعمیرات موبایل جلسه ی 2.mp4
size: 119.366
quality:720p
downloaded آموزش تعمیرات موبایل جلسه ی 2.mp4
file name: آموزش تعمیرات موبایل جلسه ی 4.mp4
size: 261.358
quality:720p
downloaded آموزش تعمیرات موبایل جلسه ی 3.mp4
file name: آموزش تعمیرات موبایل جلسه ی 5.mp4
size: 129.279
quality:720p
downloaded آموزش تعمیرات موبایل جلسه ی 1.mp4
file name: آموزش تعمیرات موبایل جلسه ی 6.mp4
size: 67.431
quality:720p
downloaded آموزش تعمیرات موبایل جلسه ی 6.mp4
file name: آموزش تعمیرات موبایل جلسه ی 8.mp4
size: 173.675
quality:720p
downloaded آموزش تعمیرات موبایل جلسه ی 5.mp4
file name: آموزش تعمیرات موبایل جلسه ی 10.mp4
size: 163.739
quality:720p
downloaded آموزش تعمیرات موبایل جلسه ی 4.mp4
file name: آموزش تعمیرات موبایل جلسه ی 11.mp4
size: 133.171
quality:720p
downloaded آموزش تعمیرات موبایل جلسه ی 8.mp4
downloaded آموزش تعمیرات موبایل جلسه

In [ ]:
def download_file(file):
    url = file[1]
    filename = file[0]
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte  
    with open('./dl_files/'+filename, 'wb') as file:
        for data in response.iter_content(block_size):
            file.write(data)
with ThreadPoolExecutor(max_workers=10) as executor:
        tqdm(executor.map(download_file, videos_list), total=len(videos_list))

In [ ]:
#get all urls and run below
url='https://www.youtube.com/watch?v=H_gAkKb30fU&t=1816s'

def progress_callback(stream: Stream, data_chunk: bytes, bytes_remaining: int) -> None:
    bytes_remaining
    pbar.update(len(data_chunk))

yt = YouTube(url, on_progress_callback=progress_callback)
video = yt.streams.get_highest_resolution() #480 if not then 720 then 360,144
print(f'file name: {video.default_filename}\nsize: {video.filesize_mb}\nquality:{video.resolution}')
#if file exists then replace
pbar = tqdm(total=video.filesize, unit="bytes")
path = video.download('./dl_files')
pbar.close()
print(f"Saved video to {path}")